In [1]:
import pandas as pd 
import numpy as np
import geopandas as gpd
from geopandas import GeoDataFrame
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from shapely.geometry import Point
from shapely.geometry import Polygon

In [2]:
# Routine to verify if the BGRI code is the original polygon
def verifyBGRIconsistency():
    notInPolygon = []
    bgriInOriginal = gdf_escolas_1CEB_BGRI['BGRI11'].value_counts().keys().tolist()
    for e in bgriInOriginal:
        if e not in gdf_BGRI_2011.index:
            notInPolygon.append(e)
    return notInPolygon

### Leitura de Ficheiros

In [3]:
gdf_BGRI_2011 = gpd.read_file('./data/BGRI_2011/CONTINENTE/BGRI11_CONT.shp')
gdf_BGRI_2011['Code_Freg'] = gdf_BGRI_2011['DTMN11'] + gdf_BGRI_2011['FR11']


In [4]:
df_correspond_nuts3 = pd.read_csv("./data/00_Base_de_dados/CODIGOS_MUNICIPIOS_NUTS3.csv",
        engine='python', delimiter= ",", dtype= {'code_mun': str})
gdf_BGRI_2011 = gdf_BGRI_2011.merge(df_correspond_nuts3[['code_mun','nome_mun']], right_on= 'code_mun', left_on = 'DTMN11', how = 'left')
gdf_BGRI_2011.drop(columns = ['code_mun'], inplace = True)

In [5]:
gdf_escolas_Portugal_1CEB = gpd.read_file('./data/00_Base_de_dados/escolas_portugal/gdf_escolas_publicas_1CEB.shp')
gdf_escolas_Portugal_1CEB = gdf_escolas_Portugal_1CEB.to_crs({'init': 'epsg:3763'})

In [6]:
gdf_escolas_Portugal_1CEB.ESCOLA.count()

3426

In [7]:
df_TIPAU = pd.read_excel("./data/00_Base_de_dados/TIPAU_Freguesia_2011.xlsx",sheet_name='SIG')
gdf_BGRI_2011 = gdf_BGRI_2011.merge(df_TIPAU[['Código','TIPAU']],
left_on='Code_Freg', right_on='Código')

In [8]:
#gdf_BGRI_2011[gdf_BGRI_2011.LUG11=='999999']

In [9]:
df_BGRI_2011_Grupo_etario = pd.read_csv("./data/00_Base_de_dados/C2011_subsec_Grupo_Etario.csv", delimiter= ',', 
dtype ={'MUNICIPIO': str, 'FREGUESIA': str, 'SECCAO': str, 'SUBSECCAO': str})

/opt/conda/envs/pandana/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [10]:
df_BGRI_Dados_gerais = pd.read_csv("./data/00_Base_de_dados/BGRI2011_PT.csv", 
sep = ';', dtype= {'GEO_COD': str},  engine = 'python')
df_BGRI_Dados_gerais['GEO_COD'] = df_BGRI_Dados_gerais['GEO_COD'].str[1:]

In [11]:
df_Accessibility_BGRI = pd.read_excel('./data/00_Base_de_dados/finalResultsAccessbility60km.xlsx', 
                                      dtype= {'Code_Freg': str, 'key_0': str})
df_Accessibility_BGRI['code_mun']=df_Accessibility_BGRI['key_0'].astype(str).str[0:4]

Calcular pop desempregada

In [12]:
df_BGRI_Dados_gerais['Pop_desempregada']= df_BGRI_Dados_gerais.loc[:,['N_IND_RESID_DESEMP_PROC_1EMPRG','N_IND_RESID_DESEMP_PROC_EMPRG']].sum(axis=1)
df_BGRI_Desempregados = df_BGRI_Dados_gerais.loc[:, ['GEO_COD', 'Pop_desempregada']]

In [13]:
df_BGRI_2011_Grupo_etario ['Pop_idade_ativa'] = df_BGRI_2011_Grupo_etario.loc[:,'15 - 19':'60 - 64'].sum(axis = 1)

In [14]:
gdf_BGRI_2011 = gdf_BGRI_2011.merge(df_BGRI_2011_Grupo_etario\
[['SUBSECCAO','Total Of CountOfIND', 'Pop_idade_ativa']][df_BGRI_2011_Grupo_etario.SEXO==0],
left_on='BGRI11', right_on='SUBSECCAO', how='left')

#what.fillna('unknown', inplace=True)

In [15]:
gdf_BGRI_2011[gdf_BGRI_2011.LUG11=='999999'].shape

(19009, 17)

In [16]:
gdf_BGRI_2011.rename(columns={"Total Of CountOfIND": "Pop_Total_2011"}, inplace =True) 

In [17]:
gdf_BGRI_2011 = gdf_BGRI_2011.merge(df_BGRI_Desempregados,
left_on='BGRI11', right_on='GEO_COD', how='left')

In [18]:
gdf_BGRI_2011['Tx_desemprego'] = gdf_BGRI_2011['Pop_desempregada']/gdf_BGRI_2011['Pop_idade_ativa']*100

In [19]:
gdf_escolas_Portugal_1CEB.head()

,DSR,DISTRITO,CONCELHO,NUTIII,CÓD. AGRUP,AGRUPAMENT,CÓD. DGEEC,CÓD. DGPGF,ESCOLA,MORADA,...,CODCONCELH,CODDISTRIT,CODNUTIII,CODGEPESED,CODDRE,ENSINOS_LI,LONG,Unnamed_ 3,CODCONCE_1,geometry
0,Norte,Viana do Castelo,Melgaço,Minho Lima,152602.0,Escolas de Melgaço,1603773,294329,Escola Básica de Melgaço,Avenida Capitão Salgueiro Maia,...,1603,16.0,10101.0,1603190.0,101.0,AB,-8.260877,Público,None,POINT (-10567.991 270827.129)
1,Norte,Viana do Castelo,Monção,Minho Lima,153023.0,Escolas de Monção,1604149,291663,"Escola Básica José Pinheiro Gonçalves, Monção",Bairro das Escolas Primárias,...,1604,16.0,10101.0,1604079.0,101.0,AB,-8.476451,Público,None,POINT (-28412.705 267350.864)
2,Norte,Viana do Castelo,Melgaço,Minho Lima,152602.0,Escolas de Melgaço,1603780,265342,"Escola Básica de Pomares, Melgaço",Pomares,...,1603,16.0,10101.0,1603190.0,101.0,AB,-8.277731,Público,None,POINT (-11970.880 265590.835)
3,Norte,Viana do Castelo,Monção,Minho Lima,153023.0,Escolas de Monção,1604542,291614,"Escola Básica de Estrada, Monção",Estrada,...,1604,16.0,10101.0,1604079.0,101.0,AB,-8.488950,Público,None,POINT (-29457.051 264948.659)
4,Norte,Viana do Castelo,Valença,Minho Lima,150587.0,"Escolas de Muralhas do Minho, Valença",1608974,294330,"Escola Básica de Friestas, Valença",Rua Amaro da Costa - Trofa de Cima,...,1608,16.0,10101.0,1608480.0,101.0,AB,-8.571300,Público,None,POINT (-36279.268 263972.853)


Fazer spatial join entres escolas e BGRI

In [20]:
gdf_escolas_1CEB_BGRI = gpd.sjoin(gdf_escolas_Portugal_1CEB, gdf_BGRI_2011[['BGRI11','geometry']], op= 'within', how='left')

In [21]:
# ----------------- Verification CODE ------------------------ #
gdf_BGRI_2011.index = gdf_BGRI_2011.BGRI11
checkBGRI = verifyBGRIconsistency()
len(checkBGRI) 

0

In [22]:
gdf_escolas_1CEB_BGRI.shape

(3426, 34)

In [23]:
gdf_escolas_Portugal_1CEB.ESCOLA.count()

3426

In [24]:
gdf_escolas_Portugal_1CEB.crs

{'init': 'epsg:3763'}

In [25]:
gdf_BGRI_2011.crs

{'init': 'epsg:3763'}

Este join está a dar pau...

In [26]:
# gdf_BGRI_2011.index = gdf_BGRI_2011.SUBSECCAO <=========== CULPADO FILHO DA PUTA
gdf_BGRI_2011['N_escolas'] = gdf_escolas_1CEB_BGRI['BGRI11'].value_counts()

In [27]:
# ----------------- Verification CODE ------------------------ #
gdf_BGRI_2011.index = gdf_BGRI_2011.BGRI11
checkBGRI = verifyBGRIconsistency()
len(checkBGRI) 

0

In [28]:
gdf_escolas_1CEB_BGRI['BGRI11'].value_counts().sum()

3426

In [29]:
gdf_BGRI_2011[gdf_BGRI_2011.N_escolas.notna()].N_escolas.sum()

3426.0

Adicionar media, max, etc. em todos os campos e para diferetns unidades (freguesia, municipio)

In [30]:
df_Accessibility_BGRI_2 = df_Accessibility_BGRI.copy()
methods = ['mean', 'max', 'min', 'std']
for i in methods:
    df_Accessibility_BGRI_2 ['Dist_freguesia_' + i] = df_Accessibility_BGRI_2.groupby('Code_Freg')['Dist_1'].\
    transform(i)

for i in methods:
    df_Accessibility_BGRI_2 ['Dist_municipio_' + i] = df_Accessibility_BGRI_2.groupby('code_mun')['Dist_1'].\
    transform(i)

df_Accessibility_BGRI_2
#df_alunos_subseccao.groupby('Code_Freg')[column_names].transform('sum'))

,id,key_0,Code_Freg,Nome_Freg,Dist_1,Dist_2,Dist_3,IQ,Diference_1_2,Diference_2_3,code_mun,Dist_freguesia_mean,Dist_freguesia_max,Dist_freguesia_min,Dist_freguesia_std,Dist_municipio_mean,Dist_municipio_max,Dist_municipio_min,Dist_municipio_std
0,152643,15110100101,151101,Sesimbra (Castelo),4160.160245,4160.160245,4160.160245,4.480000,0.000000,0.000000,1511,2057.577067,6762.897225,57.959999,1373.860782,1828.027084,6762.897225,57.959999,1376.635944
1,101852,15070500412,150705,Alto-Estanqueiro-Jardia,754.928680,754.928680,754.928680,6.413043,0.000000,0.000000,1507,931.820451,2046.949036,125.047666,461.432369,777.320428,7623.784424,13.059000,666.705818
2,101772,15070500104,150705,Alto-Estanqueiro-Jardia,827.850708,854.534563,854.534563,6.863014,26.683855,0.000000,1507,931.820451,2046.949036,125.047666,461.432369,777.320428,7623.784424,13.059000,666.705818
3,101816,15070500303,150705,Alto-Estanqueiro-Jardia,1191.199666,1227.836019,1227.836019,4.723404,36.636353,0.000000,1507,931.820451,2046.949036,125.047666,461.432369,777.320428,7623.784424,13.059000,666.705818
4,183810,03120100112,031201,Abade de Vermoim,2364.897949,2417.013916,2417.013916,5.875000,52.115967,0.000000,0312,1994.285339,2478.447998,1483.285034,358.814155,1031.359615,3594.336442,0.000000,556.178584
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201079,126427,17080400601,170804,Godim,540.600386,848.677002,60000.000000,10.313725,308.076616,59151.322998,1708,991.819543,3214.917969,103.960168,802.630890,7119.664968,17826.741089,103.960168,6137.644293
201080,126465,17080700303,170807,Peso da Régua,348.976500,792.191635,60000.000000,8.791667,443.215136,59207.808365,1708,1099.503093,2852.753784,245.679500,644.810789,7119.664968,17826.741089,103.960168,6137.644293
201081,126426,17080400505,170804,Godim,554.218239,774.685257,60000.000000,7.400000,220.467018,59225.314743,1708,991.819543,3214.917969,103.960168,802.630890,7119.664968,17826.741089,103.960168,6137.644293
201082,126424,17080400503,170804,Godim,426.961401,696.877002,60000.000000,8.660870,269.915601,59303.122998,1708,991.819543,3214.917969,103.960168,802.630890,7119.664968,17826.741089,103.960168,6137.644293


In [31]:
# df_Accessibility_BGRI_2.to_csv('./data/accessibilityAndSocialMetrics.csv')

In [32]:
df_Accessibility_BGRI_2 = df_Accessibility_BGRI_2.rename(columns={'key_0':'BGRI'})
df_Accessibility_BGRI_2.index = df_Accessibility_BGRI_2.BGRI
df_Accessibility_BGRI_2 = df_Accessibility_BGRI_2.drop(columns=['BGRI'])

In [33]:
df_Accessibility_BGRI_2.columns

Index(['id', 'Code_Freg', 'Nome_Freg', 'Dist_1', 'Dist_2', 'Dist_3', 'IQ',
       'Diference_1_2', 'Diference_2_3', 'code_mun', 'Dist_freguesia_mean',
       'Dist_freguesia_max', 'Dist_freguesia_min', 'Dist_freguesia_std',
       'Dist_municipio_mean', 'Dist_municipio_max', 'Dist_municipio_min',
       'Dist_municipio_std'],
      dtype='object')

In [34]:
gdf_BGRI_2011.columns

Index(['OBJECTID', 'DTMN11', 'FR11', 'SEC11', 'SS11', 'BGRI11', 'LUG11',
       'LUG11DESIG', 'area', 'geometry', 'Code_Freg', 'nome_mun', 'Código',
       'TIPAU', 'SUBSECCAO', 'Pop_Total_2011', 'Pop_idade_ativa', 'GEO_COD',
       'Pop_desempregada', 'Tx_desemprego', 'N_escolas'],
      dtype='object')

In [54]:
df_Accessibility_BGRI_Final = pd.merge(df_Accessibility_BGRI_2, gdf_BGRI_2011[['nome_mun','LUG11','DTMN11','Code_Freg','Pop_Total_2011','Pop_idade_ativa','Pop_desempregada', 'Tx_desemprego', 'N_escolas', 'TIPAU']], left_on=[df_Accessibility_BGRI_2.index], right_on=[gdf_BGRI_2011.index], how='outer')

In [55]:
df_Accessibility_BGRI_Final.N_escolas.sum()

3426.0

In [56]:
df_Accessibility_BGRI_Final[df_Accessibility_BGRI_Final['Dist_1'].isna()]

,key_0,id,Code_Freg_x,Nome_Freg,Dist_1,Dist_2,Dist_3,IQ,Diference_1_2,Diference_2_3,...,nome_mun,LUG11,DTMN11,Code_Freg_y,Pop_Total_2011,Pop_idade_ativa,Pop_desempregada,Tx_desemprego,N_escolas,TIPAU
201084,04070200103,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Mirandela,013810,0407,040702,9.0,2.0,1,50.000000,NaN,APR
201085,04070200108,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Mirandela,999999,0407,040702,NaN,NaN,0,NaN,NaN,APR
201086,04072900103,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Mirandela,013866,0407,040729,37.0,24.0,1,4.166667,NaN,APR
201087,04072900104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Mirandela,013866,0407,040729,28.0,17.0,0,0.000000,NaN,APR
201088,04070300103,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Mirandela,034316,0407,040703,4.0,1.0,0,0.000000,NaN,APR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255953,01010100112,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,�gueda,014422,0101,010101,11.0,9.0,0,0.000000,NaN,APR
255954,01010100107,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,�gueda,014427,0101,010101,6.0,6.0,0,0.000000,NaN,APR
255955,01010100102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,�gueda,014420,0101,010101,19.0,17.0,0,0.000000,NaN,APR
255956,01010100101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,�gueda,014420,0101,010101,9.0,4.0,0,0.000000,NaN,APR


In [57]:
df_Accessibility_BGRI_Final = df_Accessibility_BGRI_Final.drop(columns=['Dist_2', 'Dist_3', 'Diference_1_2', 'Diference_2_3'])


In [59]:
# é que já tem o Nome_Freg, mas vai buscar só os valores para as BGRI que estão na acessibilidade
# e falta também acrescentar o ['mean', 'max', 'min', 'std'], por freguesia e por município em todas as BGRI

In [60]:
df_Accessibility_BGRI_Final.to_csv('./data/accessibilityAndSocialMetrics.csv')